Python for Everyone!<br/>[Oregon Curriculum Network](http://4dsolutions.net/ocn/)

## Playing with Cyphers

### Operator Overloading in Python

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3e/EnigmaMachineLabeled.jpg/800px-EnigmaMachineLabeled.jpg" width="50%" height="50%">

The class below wraps a \_code object, a Python dict, and then uses that to implement encrypt and decrypt methods.  A simple substitution code is easy to break, based on letter frequency, presuming the original language is known.

Permutations may be chained i.e. composed, i.e. multiplied.  That which may be multiplied, may be powered as well.  The group theory notion of "inverse" makes sense in that p * ~p should give back the Identity permutation. 

Think of a "do" * "undo" such that an encrypting would pair with a decrypting, returning the original plaintext.  That's the work of the Identity permutation as well.

Cyclic notation is a clever way of representing a permutation as a tuple of tuples, whereas every permuation likewise has a unique "inversion table" defined below by Knuth.

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 10 17:39:09 2015

@author: Kirby Urner
(c) MIT License

Fun for Group Theory + Python
https://github.com/4dsolutions/Python5/blob/master/px_class.py

"""

from random import shuffle 
from string import ascii_lowercase  # all lowercase letters

class P:
    """
    A Permutation
    
    self._code: a dict, is a mapping of iterable elements 
    to themselves in any order.
    """   

    def __init__(self, 
                 the_code = None,   # direct inject
                 inv_table = None,  # construct 
                 iterable = ascii_lowercase + ' '): # default domain
        """
        start out with Identity, or directly inject the mapping as
        a dict or use an inversions table to construct the permutation
        """
        if the_code:
            self._code = the_code
            
        elif inv_table:
            values = []
            for key in sorted(inv_table, reverse=True):
                if inv_table[key] >= len(values):
                    values.append(key)
                else:
                    values.insert(inv_table[key], key)
            self._code = dict(zip(sorted(inv_table), values))
            
        elif iterable:    
            if not "__iter__" in dir(iterable):
                raise TypeError
                
            self._code = dict(zip(iterable, iterable))
        
    def shuffle(self):
        """
        return a random permutation of this permutation
        """
        # use shuffle
        # something like
        the_keys = list(self._code.keys()) # grab keys
        shuffle(the_keys)  # shuffles other one
        newP = P()
        newP._code = dict(zip(self._code.keys(), the_keys))
        return newP
        
    def encrypt(self, plain):
        """
        turn plaintext into cyphertext using self._code
        """
        output = ""  # empty string
        for c in plain:
            output = output + self._code.get(c, c) 
        return output
            
    def decrypt(self, cypher):
        """
        Turn cyphertext into plaintext using ~self
        """
        reverse_P = ~self  # invert me!
        output = ""
        for c in cypher:
            output = output + reverse_P._code.get(c, c)
        return output
 
    def __getitem__(self, key):
        return self._code.get(key, None)
               
    def __repr__(self):
        return "P class: " + str(tuple(self._code.items())[:3]) + "..."

    def cyclic(self):
        """
        cyclic notation, a compact view of a group
        """
        output = []
        the_dict = self._code.copy()
        while the_dict:
            start = tuple(the_dict.keys())[0]
            the_cycle = [start]
            the_next = the_dict.pop(start)
            while the_next != start:
                the_cycle.append(the_next)
                the_next = the_dict.pop(the_next)
            output.append(tuple(the_cycle))
        return tuple(output)

    def __mul__(self, other): 
        """
        look up my keys to get values that serve
        as keys to get others "target" values
        """
        new_code = {}
        for c in self._code:  # going through my keys
            target = other._code[ self._code[c] ]
            new_code[c] = target
        new_P = P(' ') 
        new_P._code = new_code
        return new_P
        
    def __truediv__(self, other):
        return self * ~other
                
    def __pow__(self, exp):
        """
        multiply self * self the right number of times
        """
        if exp == 0:
            output = P()
        else:
            output = self

        for x in range(1, abs(exp)):
            output *= self
        
        if exp < 0:
            output = ~output
            
        return output

    def __call__(self, s): 
        """
        another way to encrypt
        """
        return self.encrypt(s)  

    def __invert__(self):
        """
        create new P with reversed dict
        """
        newP = P(' ')
        newP._code = dict(zip(self._code.values(), self._code.keys()))
        return newP
        
    def __eq__(self, other):
        """
        are these permutation the same?  
        Yes if self._code == other._code
        """
        return self._code == other._code
        
    def inversion_table(self):
        invs = {}
        invP = ~self
        keys = sorted(self._code)
        for key in keys:
            x = invP[key] # position of key
            cnt = 0
            for left_of_key in keys: # in order up to position
                if left_of_key == x: # none more to left
                    break
                if self._code[left_of_key] > key:
                    cnt += 1
            invs[key] = cnt
        return invs

In this first test, lets make sure that any random permutation, times its inverse, gives the Identity permutation...

In [2]:
p = P() # identity permutation
new_p = p.shuffle()
inv_p = ~new_p 
try:
    assert p == inv_p * new_p   # should be True
    print("First Test Succeeds")
except AssertionError:
    print("First Test Fails")

First Test Succeeds


Since it makes sense to both power, and invert, we may raise permutations to a negative power. (p \*\* -1) is another way of saying "give me the inverse"...

In [4]:
p = P().shuffle()
try:
    assert p ** -1 == ~p
    assert p ** -2 == ~(p * p)
    assert p ** -2 == (~p * ~p)
    print("Second Test Succeeds")
except AssertionError:
    print("Second Test Fails")

Second Test Succeeds


Since Permutation p will be able to compute it's own inverse, decrypting the cyphertext is a snap...

In [6]:
p = P().shuffle()
s = "able was i ere i saw elba"
c = p(s)
print("Plain:  ", s)
print("Cipher: ", c)
try:
    assert p.decrypt(c) == s
    print("Third Test Succeeds")
except AssertionError:
    print("Third Test Fails")

Plain:   able was i ere i saw elba
Cipher:  dvq bwdnbpb e bpbndwb qvd
Third Test Succeeds


"The inversion table *B1 B2 ... Bn* of the permutation *A1, A2 ... An* is obtained by letting *Bj* be the number of elements to the left of *j* that are greater than *j*." -- Donald E. Knuth

![Knuth quote](https://c8.staticflickr.com/9/8408/29639707343_ab1df95f9e_z.jpg)

In [3]:
knuth = {1:5, 2:9, 3:1, 4:8, 5:2, 6:6, 7:4, 8:7, 9:3} # vol 3 pg. 12
expected = {1:2, 2:3, 3:6, 4:4, 5:0, 6:2, 7:2, 8:1, 9:0} # Ibid
k = P(the_code=knuth)
try: 
    assert k.inversion_table() == expected
    print("Fourth Test Succeeds")
except AssertionError:
    print("Fourth Test Fails")

Fourth Test Succeeds


Lets make sure that, given the derived inversion table for the Permutation named *knuth*, we're able to regenerate *knuth* upon initialization...

In [11]:
p = P(inv_table = expected)
try: 
    assert p == k
    print("Fifth Test Succeeds")
except AssertionError:
    print("Fifth Test Fails")

Fifth Test Succeeds


Finally, given any random permutation, lets make sure we're able to reconstruct it from its inversion table.  The only difference from the test above is we're using a random permutation versus the one given on page 12 in volume 3 of *The Art of Computer Programming*.

In [9]:
p = P().shuffle()
inv = p.inversion_table()
print("Perm:", p._code)
print("Inv table:", inv)
new_p = P(inv_table = inv)
try: 
    assert p == new_p
    print("Sixth Test Succeeds")
except AssertionError:
    print("Sixth Test Fails") 

Perm: {'a': 'j', 'r': ' ', 'h': 'v', 'o': 'f', 'p': 'l', 'q': 'e', 'j': 'h', 'd': 'q', 'f': 'i', 'w': 's', 's': 'g', 'x': 't', 'v': 'x', 'i': 'a', 'u': 'z', 'k': 'y', ' ': 'n', 'm': 'w', 'l': 'r', 'g': 'm', 't': 'k', 'z': 'u', 'y': 'd', 'b': 'c', 'n': 'b', 'e': 'p', 'c': 'o'}
Inv table: {'a': 9, 'w': 1, 'h': 8, 'o': 0, 's': 5, 'p': 1, 'q': 0, 'j': 1, 'd': 21, 'k': 10, 'r': 2, 'e': 14, 'x': 2, 'v': 0, 'i': 5, 'u': 5, 'f': 12, ' ': 18, 'm': 4, 'l': 9, 'g': 13, 't': 5, 'z': 0, 'y': 0, 'b': 13, 'n': 0, 'c': 2}
Sixth Test Succeeds
